In [1]:
import os
import numpy as np
import math
import librosa
import tflearn

/home/sandimulyadi/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/sandimulyadi/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/sandimulyadi/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/sandimulyadi/.local/lib/python3.6/site-packages/tensorflow/python/f

In [2]:
yLength = 0
MFCCS = list()
LABELS = list()
_MFCCS = list()
_LABELS = list()
TRAIN_ITERS = 100

PATH_TRAIN = './dataset/train/'
DATA_TRAIN = [x for x in os.listdir(PATH_TRAIN) if x.endswith('.wav')]
PATH_VALID = './dataset/valid/'
DATA_VALID = [x for x in os.listdir(PATH_VALID) if x.endswith('.wav')]
PATH_TESTS = './dataset/test/'
DATA_TESTS = [x for x in os.listdir(PATH_TESTS) if x.endswith('.wav')]

In [3]:
for x in DATA_TRAIN:
    y, sr = librosa.load(PATH_TRAIN + x, sr=None, mono=True)
    emphasized_signal = np.append(y[0], y[1:] - 0.97 * y[:-1])
    if len(emphasized_signal) > yLength:
        yLength = len(emphasized_signal)
        
for x in DATA_VALID:
    y, sr = librosa.load(PATH_VALID + x, sr=None, mono=True)
    emphasized_signal = np.append(y[0], y[1:] - 0.97 * y[:-1])
    if len(emphasized_signal) > yLength:
        yLength = len(emphasized_signal)
        
for x in DATA_TESTS:
    y, sr = librosa.load(PATH_TESTS + x, sr=None, mono=True)
    emphasized_signal = np.append(y[0], y[1:] - 0.97 * y[:-1])
    if len(emphasized_signal) > yLength:
        yLength = len(emphasized_signal)
        
print('Length of vectorized data is %s' % yLength)

Length of vectorized data is 1100160


In [4]:
def VektorReshape(param):    
    if len(param) < yLength:
        _ceil = math.ceil((yLength - len(param)) / 2)
        _floor = math.floor((yLength - len(param)) / 2)
        return np.pad(param, (_floor, _ceil), 'constant', constant_value=(0))
    
    return param

def LABEL_AL_FATIHA(filename):
    ayah = int(filename[5])
    
    return np.eye(8)[ayah]

In [12]:
y, sr = librosa.load(PATH_TRAIN + '001000_Abdul_Basit_Mujawwad_128kbps.wav', sr=None, mono=True)
emphasized_signal = np.append(y[0], y[1:] - 0.97 * y[:-1])
print(emphasized_signal)
b = np.pad(emphasized_signal, (2,3), mode='constant', constant_values=0)
print(b)

[ 1.5258789e-05 -7.5836186e-05  1.5075684e-04 ...  1.4526362e-04
  3.0105584e-04  1.1047360e-04]
[0.0000000e+00 0.0000000e+00 1.5258789e-05 ... 0.0000000e+00 0.0000000e+00
 0.0000000e+00]


In [11]:
for x in DATA_TRAIN:
    y, sr = librosa.load(PATH_TRAIN + x, sr=None, mono=True)
    emphasized_signal = np.append(y[0], y[1:] - 0.97 * y[:-1])
    mfcc = librosa.feature.mfcc(emphasized_signal, sr)
    LABELS.append(LABEL_AL_FATIHA(x))
    MFCCS.append(np.array(mfcc))
        
for x in DATA_VALID:
    y, sr = librosa.load(PATH_VALID + x, sr=None, mono=True)
    emphasized_signal = np.append(y[0], y[1:] - 0.97 * y[:-1])
    mfcc = librosa.feature.mfcc(emphasized_signal, sr)
    _LABELS.append(LABEL_AL_FATIHA(x))
    _MFCCS.append(np.array(mfcc))

In [12]:
batchs = list()
for i in range(1, len(MFCCS)+1):
    if len(MFCCS) % i == 0:
        batchs.append(i)
print(batchs)
BATCH = int(input("Please choose a batch size: "))
if BATCH not in batchs:
    print("Choosen batch size: %d" % len(MFCCS))

net = tflearn.input_data([None, 20, yLength])
net = tflearn.lstm(net, 128, dropout=0.8)
net = tflearn.fully_connected(net, 8, activation='softmax')
net = tflearn.regression(net, optimizer='adam', learning_rate=0.0001, loss='categorical_crossentropy')

model = tflearn.DNN(net, tensorboard_verbose=0)
for i in range(TRAIN_ITERS):
    model.fit(MFCCS, LABELS, n_epoch=10, validation_set=(_MFCCS, _LABELS), show_metric=True, batch_size=BATCH)
model.save("tflearn.lstm_with_padding_%d.model" % BATCH)

[1, 2, 3, 6, 9, 17, 18, 34, 51, 102, 153, 306]
Please choose a batch size: 17


IndexError: list index out of range